# Práctica 1b
Hacer un programa para dibujar las siguientes primitivas con el ratón
- Líneas
- Rectángulos
- Círculos
- Especificar el color en RGB de las primitivas
- Definir el grosor de las líneas

In [5]:
import cv2 as cv
import numpy as np

# --- Cargar imagen ---
file = 'images/eii.png'
image = cv.imread(file)
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")

canvas = image.copy()
drawing = False
ix, iy = -1, -1

# --- Ventana principal ---
cv.namedWindow("Editor")

# --- Funciones de trackbars ---
def nothing(x):
    pass

# Trackbar para la figura (0=linea, 1=rect, 2=circle)
cv.createTrackbar("Figura", "Editor", 0, 2, nothing)
# Trackbars para el color BGR
cv.createTrackbar("B", "Editor", 0, 255, nothing)
cv.createTrackbar("G", "Editor", 0, 255, nothing)
cv.createTrackbar("R", "Editor", 255, 255, nothing)
# Trackbar para grosor
cv.createTrackbar("Grosor", "Editor", 2, 10, nothing)

# --- Función de dibujo con ratón ---
def draw(event, x, y, flags, param):
    global ix, iy, drawing, canvas, image

    figura = cv.getTrackbarPos("Figura", "Editor")
    b = cv.getTrackbarPos("B", "Editor")
    g = cv.getTrackbarPos("G", "Editor")
    r = cv.getTrackbarPos("R", "Editor")
    color = (b, g, r)
    thickness = cv.getTrackbarPos("Grosor", "Editor")

    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv.EVENT_MOUSEMOVE and drawing:
        temp = image.copy()
        if figura == 0:   # línea
            cv.line(temp, (ix, iy), (x, y), color, thickness)
        elif figura == 1: # rectángulo
            cv.rectangle(temp, (ix, iy), (x, y), color, thickness)
        elif figura == 2: # círculo
            radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
            cv.circle(temp, (ix, iy), radius, color, thickness)
        canvas = temp

    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        if figura == 0:
            cv.line(image, (ix, iy), (x, y), color, thickness)
        elif figura == 1:
            cv.rectangle(image, (ix, iy), (x, y), color, thickness)
        elif figura == 2:
            radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
            cv.circle(image, (ix, iy), radius, color, thickness)
        canvas = image.copy()

# Asociar el callback
cv.setMouseCallback("Editor", draw)

print("\n👉 Usa los sliders para cambiar figura, color y grosor.")
print("Figura: 0=Línea, 1=Rectángulo, 2=Círculo")
print("ESC para salir.")

# Loop principal
while True:
    cv.imshow("Editor", canvas)
    if cv.waitKey(1) & 0xFF == 27:  # ESC
        break

cv.destroyAllWindows()



👉 Usa los sliders para cambiar figura, color y grosor.
Figura: 0=Línea, 1=Rectángulo, 2=Círculo
ESC para salir.


In [1]:
import cv2 as cv
import numpy as np
import threading
import tkinter as tk
from tkinter import simpledialog

# --- Imagen base ---
file = "images/eii.png"
image = cv.imread(file)
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")
canvas = image.copy()

# --- Variables globales ---
drawing = False
ix, iy = -1, -1
shape = "line"
color = (0, 0, 255)  # rojo
thickness = 2

# --- Función de dibujo ---
def draw(event, x, y, flags, param):
    global ix, iy, drawing, canvas, image, shape, color, thickness

    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv.EVENT_MOUSEMOVE and drawing:
        temp = image.copy()
        if shape == "line":
            cv.line(temp, (ix, iy), (x, y), color, thickness)
        elif shape == "rect":
            cv.rectangle(temp, (ix, iy), (x, y), color, thickness)
        elif shape == "circle":
            radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
            cv.circle(temp, (ix, iy), radius, color, thickness)
        canvas = temp

    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        if shape == "line":
            cv.line(image, (ix, iy), (x, y), color, thickness)
        elif shape == "rect":
            cv.rectangle(image, (ix, iy), (x, y), color, thickness)
        elif shape == "circle":
            radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
            cv.circle(image, (ix, iy), radius, color, thickness)
        canvas = image.copy()

# --- Funciones de UI ---
def set_shape(s):
    global shape
    shape = s
    print(f"✔ Figura cambiada a {s}")

def set_color(c):
    global color
    if c == "rojo":
        color = (0, 0, 255)
    elif c == "verde":
        color = (0, 255, 0)
    elif c == "azul":
        color = (255, 0, 0)
    print(f"✔ Color cambiado a {c}")

def set_thickness():
    global thickness
    try:
        val = simpledialog.askinteger("Grosor", "Introduce grosor en píxeles:", minvalue=1, maxvalue=20)
        if val:
            thickness = val
            print(f"✔ Grosor cambiado a {thickness}")
    except:
        pass

# --- Ventana Tkinter ---
def tk_ui():
    root = tk.Tk()
    root.title("Opciones de Dibujo")

    # Botones de figura
    tk.Label(root, text="Figura:").pack()
    tk.Button(root, text="Línea", command=lambda: set_shape("line")).pack(fill="x")
    tk.Button(root, text="Rectángulo", command=lambda: set_shape("rect")).pack(fill="x")
    tk.Button(root, text="Círculo", command=lambda: set_shape("circle")).pack(fill="x")

    # Botones de color
    tk.Label(root, text="Color:").pack()
    tk.Button(root, text="Rojo", bg="red", command=lambda: set_color("rojo")).pack(fill="x")
    tk.Button(root, text="Verde", bg="green", command=lambda: set_color("verde")).pack(fill="x")
    tk.Button(root, text="Azul", bg="blue", fg="white", command=lambda: set_color("azul")).pack(fill="x")

    # Grosor
    tk.Button(root, text="Cambiar grosor", command=set_thickness).pack(fill="x")

    root.mainloop()

# Lanzar Tkinter en un hilo paralelo
threading.Thread(target=tk_ui, daemon=True).start()

# --- OpenCV ---
cv.namedWindow("Editor")
cv.setMouseCallback("Editor", draw)

print("\n👉 Dibuja con el ratón en la ventana 'Editor'. Usa la ventana Tkinter para cambiar opciones.")
print("Pulsa ESC para salir.")

while True:
    cv.imshow("Editor", canvas)
    if cv.waitKey(1) & 0xFF == 27:  # ESC
        break

cv.destroyAllWindows()



👉 Dibuja con el ratón en la ventana 'Editor'. Usa la ventana Tkinter para cambiar opciones.
Pulsa ESC para salir.
✔ Figura cambiada a line
✔ Figura cambiada a rect
✔ Grosor cambiado a 19
✔ Color cambiado a verde
✔ Color cambiado a azul
